In [ ]:
'''
Arquivos:
- Estabelecer a forma de guardar os arquivos conforme necessidade da CNN

Funções:
- Função de importação de arquivo com as seguintes colunas:
    Open High Low Close Volume + Indicadores
  Esta mesma função fará os índices das linhas serem as datas
- Função de escolha do intervalo de dias a terem o gráfico montado
- Função que gera o gráfico de candlestick com volume e indicadores

'''

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
from datetime import datetime
from datetime import timedelta
from PIL import Image


In [ ]:
'''
Importa os dados do ativo e renomeia as colunas
'''

asset = 'BTOW3'
time_frame = '1d'
file_name = asset + '_' + time_frame + '.xlsx'
path = '../collected_ data/' + file_name

inplace = True
df_input = pd.read_excel(path)
df_input.dropna(inplace = inplace)
df_input.set_index('Data', inplace = inplace)
df_input.sort_index(axis = 0, inplace = inplace)
df_input.rename(inplace = inplace, columns = {
    'Abertura': 'Open',
    'Máxima': 'High',
    'Mínima': 'Low',
    'Fechamento': 'Close',
    'Volume Financeiro': 'Volume',
    'Média Móvel E [9]': 'MME009',
    'Média Móvel A [21]': 'MMA021',
    'Média Móvel A [200]': 'MMA200',
    'Média Móvel A [51]': 'MMA051',
    'Média Móvel A [80]': 'MMA080',
    'Média Móvel A [400]': 'MMA400',
    'Média Móvel E [400]': 'MME400'})

# df_input.head()


In [ ]:
'''
Cria reordena as colunas dos indicadores em ordem alfabética
'''

df_candle = df_input[['Open', 'High', 'Low', 'Close', 'Volume']]
df_ind = df_input.reindex(sorted(df_input.columns[5:]), axis = 1)
df = pd.concat([df_candle, df_ind], axis=1)

# df.head()


In [ ]:
'''
Verifica se o dataframe reordenado é igual ao original
'''

columns = list(df.columns)
eq = True
for col in columns:
   eq = eq*df[col].equals(df_input[col])
if eq:
    print("Data Frame válido.")
else:
    print("Data Frame inválido!!!!")
    

In [ ]:
'''
Aprender a gerar os gráficos para então iterar os intervalos de data
'''

def date_delta (start_date, delta): # Função sem uso por enquanto
    end_date = datetime.strptime(start_date, '%Y-%m-%d') + timedelta(days = delta)
    return end_date.strftime('%Y-%m-%d')


def plot_chart(df, start_date, delta, add, save_name):
    start_index = df.index.get_loc(start_date)
    end_index = start_index + delta

    df_plot = df.iloc[start_index:end_index,:]
    erase_color = 'black'

    mc = mpf.make_marketcolors(up = 'g',down = 'r',
                               wick = 'inherit',
                               edge = 'inherit',
                               volume = 'grey')
    s = mpf.make_mpf_style(marketcolors = mc,
                           facecolor = erase_color,
                           edgecolor = erase_color,
                           figcolor = erase_color)
    addplot = mpf.make_addplot(df_plot[add])

    plot = mpf.plot(df_plot,
                    type = 'candle',
                    volume = True,
                    style = s,
                    addplot = addplot,
                    savefig = '../generated_data/plot.png')

    uncropped_plot = Image.open('../generated_data/plot.png')
    box = (uncropped_plot.size[0]*0.202,
           uncropped_plot.size[1]*0.11,
           uncropped_plot.size[0]*0.88,
           uncropped_plot.size[1]*0.85)

    cropped_plot = uncropped_plot.crop(box).save('../generated_data/' + save_name + '.png')
    return
    

In [ ]:
'''
Chamamento das funções de geração dos gráficos
Implementar o loop for
'''

ind_columns = list(df.columns)[5:9]
start_date = '2020-01-02' # Captura exceção KeyError e passa para o próximo loop
delta = 120
save_name = asset + '.' + start_date # Adicionar a classe
plot_chart(df, start_date, delta, ind_columns, save_name)


In [ ]:
'''
Escrever função que organiza aleatoriamento os arquivos em pastas
Pastas: train, valid e test

Cada classe terá o mesmo número de exemplos. Avaliar possibilidade de copiar ao invés de mover.
Dica: files_distribution = [[long, n_long]],
                            [short, n_short],
                            [do_nothing, n_do_nothin]]
      for class in files_distribution:
          for file in random.sample(glob.glob(class[0] + '.*'), class[1]):
              shutil.move(file, '/charts/' + class[0])
'''